In [2]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import glob, os
import random


import tensorflow as tf
from tensorflow import keras

physical_devices = tf.config.list_physical_devices('GPU') #To list the available amount of GPUs
print(physical_devices)


try:
    for kgpu in range(len(physical_devices)):
        tf.config.experimental.set_memory_growth(physical_devices[kgpu], True)  #Set if memory growth should be enabled for a PhysicalDevice
except:
    # Invalid device or cannot modify virtual devices once initialized.
    pass

[]


In [10]:
# in/out variable lists for v1

vars_mli = ['state_t','state_q0001','state_ps','pbuf_SOLIN', 'pbuf_LHFLX', 'pbuf_SHFLX']
vars_mlo = ['ptend_t','ptend_q0001','cam_out_NETSW','cam_out_FLWDS','cam_out_PRECSC','cam_out_PRECC','cam_out_SOLS','cam_out_SOLL','cam_out_SOLSD','cam_out_SOLLD']

In [11]:
#To noralize (with precomputed normalization factors)

path = '/gpfswork/rech/psl/upu87pm/ClimSim'

mli_mean = xr.open_dataset(path + '/preprocessing/normalizations/inputs/input_max.nc')
mli_min = xr.open_dataset(path + '/preprocessing/normalizations/inputs/input_min.nc')
mli_max = xr.open_dataset(path + '/preprocessing/normalizations/inputs/input_max.nc')
mlo_scale = xr.open_dataset(path + '/preprocessing/normalizations/outputs/output_scale.nc')



In [12]:
#function defined in climsim_utils.data_utils, I should be able to import it 


def load_nc_dir_with_generator(filelist:list):
    '''
        This function works as a dataloader when training the emulator with raw netCDF files.
        This can be used as a dataloader during training or it can be used to create entire datasets. !!
        When used as a dataloader for training, I/O can slow down training considerably.
        This function also normalizes the data.
        mli corresponds to input
        mlo corresponds to target
        '''
    def gen():
        for file in filelist:
            
            # read mli
            ds = xr.open_dataset(file, engine='netcdf4')
            ds = ds[vars_mli]                             
            
            # read mlo
            dso = xr.open_dataset(file.replace('.mli.','.mlo.'), engine='netcdf4') 
            
            # make mlo variales: ptend_t and ptend_q0001
            dso['ptend_t'] = (dso['state_t'] - ds['state_t'])/1200 # T tendency [K/s]
            dso['ptend_q0001'] = (dso['state_q0001'] - ds['state_q0001'])/1200 # Q tendency [kg/kg/s]
            dso = dso[vars_mlo]
            
            # normalizatoin, scaling
            ds = (ds-mli_mean)/(mli_max-mli_min)
          
            dso = dso*mlo_scale

            # stack
            #ds = ds.stack({'batch':{'sample','ncol'}})
            ds = ds.stack({'batch':{'ncol'}})
            ds = ds.to_stacked_array("mlvar", sample_dims=["batch"], name='mli')
            #dso = dso.stack({'batch':{'sample','ncol'}})
            dso = dso.stack({'batch':{'ncol'}})
            dso = dso.to_stacked_array("mlvar", sample_dims=["batch"], name='mlo')
            
            yield (ds.values, dso.values)

    return tf.data.Dataset.from_generator(
        gen,
        output_types=(tf.float64, tf.float64),
        output_shapes=((None,124),(None,128))
    )

In [13]:
path_data = '/gpfsscratch/rech/yvd/upu87pm/lr'

shuffle_buffer=12 

# for training

# # First 5 days of each month for the first 6 years
# f_mli1 = glob.glob('/pscratch/sd/s/sungduk/hugging/E3SM-MMF_ne4/train/*/E3SM-MMF.mli.000[123456]-*-0[12345]-*.nc')
# f_mli2 = glob.glob('/pscratch/sd/s/sungduk/hugging/E3SM-MMF_ne4/train/*/E3SM-MMF.mli.0007-01-0[12345]-*.nc')
# f_mli = [*f_mli1, *f_mli2]


# every 2th sample 
f_mli1 = glob.glob(path_data +'/0001/E3SM-MMF.mli.000[1]-10-0[123456789]-*.nc') #first year 10 first days of october for training (list of filename)
f_mli2 = glob.glob(path_data +'/0001/E3SM-MMF.mli.0001-10-1[0123456789]-*.nc')  #first year 10 second days of october for training

f_mli = sorted([*f_mli1, *f_mli2]) #the * is to have at the end just a list of file <=> sorted(f_mli1+f_mli2), no idea why they sort

random.shuffle(f_mli)  #I don't see the point of shuffle here
f_mli = f_mli[::2]

# # debugging
# f_mli = f_mli[0:72*5]

random.shuffle(f_mli)
print(f'[TRAIN] Total # of input files: {len(f_mli)}')
print(f'[TRAIN] Total # of columns (nfiles * ncols): {len(f_mli)*384}')
tds = load_nc_dir_with_generator(f_mli)
tds = tds.unbatch()
tds = tds.shuffle(buffer_size=shuffle_buffer, reshuffle_each_iteration=True) #lot of shuffle
tds = tds.prefetch(buffer_size=4) # in realtion to the batch size    these 3 last line seems useless since we'll do it at each epoch

# for validation

# # First 5 days of each month for the following 2 years
# f_mli1 = glob.glob('/pscratch/sd/s/sungduk/hugging/E3SM-MMF_ne4/train/*/E3SM-MMF.mli.0007-0[23456789]-0[12345]-*.nc')
# f_mli2 = glob.glob('/pscratch/sd/s/sungduk/hugging/E3SM-MMF_ne4/train/*/E3SM-MMF.mli.0007-1[012]-0[12345]-*.nc')
# f_mli3 = glob.glob('/pscratch/sd/s/sungduk/hugging/E3SM-MMF_ne4/train/*/E3SM-MMF.mli.000[89]-*-0[12345]-*.nc')
# f_mli_val = [*f_mli1, *f_mli2, *f_mli3]

# every 2th sample
f_mli1 = glob.glob(path_data+'/0002/10/E3SM-MMF.mli.0002-10-0[123456789]-*.nc')
f_mli2 = glob.glob(path_data+'/0002/10/E3SM-MMF.mli.0002-10-1[012345678]-*.nc')
f_mli3 = glob.glob(path_data+'/0002/10/E3SM-MMF.mli.000[2]-10-2[012345678]-*.nc')
f_mli_val = sorted([*f_mli1, *f_mli2, *f_mli3])
f_mli_val = f_mli_val[::10]

# # debugging
# f_mli_val = f_mli_val[0:72*5]

random.shuffle(f_mli_val)
print(f'[VAL] Total # of input files: {len(f_mli_val)}')
print(f'[VAL] Total # of columns (nfiles * ncols): {len(f_mli_val)*384}')
tds_val = load_nc_dir_with_generator(f_mli_val)
tds_val = tds_val.shuffle(buffer_size=shuffle_buffer, reshuffle_each_iteration=True)
tds_val = tds_val.prefetch(buffer_size=4) # in realtion to the batch size

#list(tds)
# for count_batch in tds.repeat().batch(10).take(1):
#     print(count_batch[0].numpy())
#count_batch[0].shape

[TRAIN] Total # of input files: 684
[TRAIN] Total # of columns (nfiles * ncols): 262656
[VAL] Total # of input files: 195
[VAL] Total # of columns (nfiles * ncols): 74880


In [14]:
tf.config.list_physical_devices('GPU')

[]

In [15]:
#Model given by ClimSim (not the one they are using in the paper)
#No idea what is this one if it is not juste a test example



# model params
input_length = 2*60 + 4  #temperature and humidity have the vertical dimension
output_length_lin  = 2*60  #only temperature and moisting tendency

output_length_relu = 8
output_length = output_length_lin + output_length_relu
n_nodes = 512

# constrcut a model
input_layer    = keras.layers.Input(shape=(input_length,), name='input')
hidden_0       = keras.layers.Dense(n_nodes, activation='relu')(input_layer)
hidden_1       = keras.layers.Dense(n_nodes, activation='relu')(hidden_0)
output_pre     = keras.layers.Dense(output_length, activation='elu')(hidden_1)
output_lin     = keras.layers.Dense(output_length_lin,activation='linear')(output_pre)
output_relu    = keras.layers.Dense(output_length_relu,activation='relu')(output_pre)
output_layer   = keras.layers.Concatenate()([output_lin, output_relu])

model = keras.Model(input_layer, output_layer, name='Emulator')
model.summary()

# compile
model.compile(optimizer=keras.optimizers.Adam(), #optimizer=keras.optimizers.Adam(learning_rate=clr),
              loss='mse',
              metrics=['mse','mae','accuracy'])

Model: "Emulator"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 124)]        0           []                               
                                                                                                  
 dense_5 (Dense)                (None, 512)          64000       ['input[0][0]']                  
                                                                                                  
 dense_6 (Dense)                (None, 512)          262656      ['dense_5[0][0]']                
                                                                                                  
 dense_7 (Dense)                (None, 128)          65664       ['dense_6[0][0]']                
                                                                                           

In [16]:
# callbacks
# a. tensorboard
tboard_callback = keras.callbacks.TensorBoard(log_dir = './logs_tensorboard',
                                              histogram_freq = 1,)

# b. checkpoint
filepath_checkpoint = 'saved_model/best_model_proto.h5'
checkpoint_callback = keras.callbacks.ModelCheckpoint(filepath=filepath_checkpoint,
                                                            save_weights_only=False,
                                                            monitor='val_mse',
                                                            mode='min',
                                                            save_best_only=True)

# c. csv logger
filepath_csv = 'csv_logger.txt'
csv_callback = keras.callbacks.CSVLogger(filepath_csv, separator=",", append=True)

my_callbacks= [tboard_callback, checkpoint_callback, csv_callback]

# !mkdir logs_tensorboard
# !mkdir saved_model

In [ ]:
# Manually shuffling the order of input files.
# "tds = tds.shuffle(buffer_size=<global>, reshuffle_each_iteration=True)" is possible,
# however, it is slow.
# So employing global shuffle (by file names) + local shuffle (using .shuffle).

N_EPOCHS = 2
shuffle_buffer = 12*384 #ncol=384
batch_size= 96 # 384/4

n=0
while n < N_EPOCHS:
    random.shuffle(f_mli)
    tds = load_nc_dir_with_generator(f_mli) # global shuffle by file names
    tds = tds.unbatch()
 # local shuffle by elements    tds = tds.shuffle(buffer_size=shuffle_buffer, reshuffle_each_iteration=False)
    tds = tds.batch(batch_size)
    tds = tds.prefetch(buffer_size=int(shuffle_buffer/384)) # in realtion to the batch size

    random.shuffle(f_mli_val)
    tds_val = load_nc_dir_with_generator(f_mli_val)
    tds_val = tds_val.unbatch()
    tds_val = tds_val.shuffle(buffer_size=shuffle_buffer, reshuffle_each_iteration=False)
    tds_val = tds_val.batch(batch_size)
    tds_val = tds_val.prefetch(buffer_size=int(shuffle_buffer/384))
    
    print(f'Epoch: {n+1}')
    model.fit(tds, validation_data=tds_val, callbacks=my_callbacks)
    #model.fit(tds, validation_data=tds_val)
    n+=1
    
model.save('my_model')


Epoch: 1
   2736/Unknown - 218s 80ms/step - loss: 0.0058 - mse: 0.0058 - mae: 0.0321 - accuracy: 0.9518

In [2]:
new_model = tf.keras.models.load_model('my_model')


In [3]:
new_model.summary()


Model: "Emulator"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 124)]        0           []                               
                                                                                                  
 dense (Dense)                  (None, 512)          64000       ['input[0][0]']                  
                                                                                                  
 dense_1 (Dense)                (None, 512)          262656      ['dense[0][0]']                  
                                                                                                  
 dense_2 (Dense)                (None, 128)          65664       ['dense_1[0][0]']                
                                                                                           

In [3]:
fn_x_input = '/gpfsscratch/rech/psl/upu87pm/preprocessed_data/scoring_input.npy'
fn_x_target = '/gpfsscratch/rech/psl/upu87pm/preprocessed_data/scoring_target.npy'

input_set = x_true = np.load(fn_x_input).astype(np.float64)
target_set = x_true = np.load(fn_x_target).astype(np.float64)


print(input_set.shape)
print(target_set.shape)


(1681920, 124)
(1681920, 128)


In [13]:

mini_set_input = np.array([input_set[i,:] for i in range(1000)])


f_input_save =  '../data/npy_data/first_test_input_set.npy'



f_save = 'npy_data/prediction/first_test_prediction.npy'
np.save(f_input_save, mini_set_input)
